In [1]:
import pprint
import tensorflow as tf
from tfx import v1 as tfx
import pandas as pd
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [2]:
from dotenv import find_dotenv, load_dotenv
import tempfile
import os
import absl
import urllib

In [3]:
#thist the root directory for your your TFX pip package installation
_tfx_root = tfx.__path__[0]
_autoencoder_root = os.path.join(_tfx_root, 'examples\\autoencoder_pipeline')
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model\\autoencoder')     
_metadata = os.path.join(_tfx_root, 'examples\\metadata')
absl.logging.set_verbosity(absl.logging.INFO)


In [4]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')

_data_filepath = os.path.join(_data_root, "data.csv")



In [5]:
load_dotenv()
df = pd.read_csv(os.getenv('UNBALANCED_CREDIT_DATA_CSV'))
df = df[df['Class']== 0]
df = df.drop(['Class','Time'],axis=1)
df.to_csv(_data_filepath,index = False)

In [7]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

In [8]:
# EXAMPLE GEN
from tfx.proto import example_gen_pb2

# output = tfx.proto.Output(
#     split_config = example_gen_pb2.SplitConfig(splits = [
#         tfx.proto.SplitConfig.Split(name = 'train', hash_buckets= 3),
#         tfx.proto.SplitConfig.Split(name = 'eval', hash_buckets= 1),
#     ])
# )
example_gen = tfx.components.CsvExampleGen(input_base = _data_root)
context.run(example_gen, enable_cache = True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data C:\Users\w10\AppData\Local\Temp\tfx-data2_sczfq0\* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [9]:
# Let's examine the output artifacts of ExampleGen. 
# This component produces two artifacts, training examples and evaluation examples:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] C:\Users\w10\AppData\Local\Temp\tfx-interactive-2022-06-09T23_39_36.874953-zuupnqn2\CsvExampleGen\examples\1


In [10]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Amount"
    value {
      float_list {
        value: 378.6600036621094
      }
    }
  }
  feature {
    key: "V1"
    value {
      float_list {
        value: -1.358354091644287
      }
    }
  }
  feature {
    key: "V10"
    value {
      float_list {
        value: 0.20764286816120148
      }
    }
  }
  feature {
    key: "V11"
    value {
      float_list {
        value: 0.6245014667510986
      }
    }
  }
  feature {
    key: "V12"
    value {
      float_list {
        value: 0.06608368456363678
      }
    }
  }
  feature {
    key: "V13"
    value {
      float_list {
        value: 0.7172927260398865
      }
    }
  }
  feature {
    key: "V14"
    value {
      float_list {
        value: -0.16594591736793518
      }
    }
  }
  feature {
    key: "V15"
    value {
      float_list {
        value: 2.34586501121521
      }
    }
  }
  feature {
    key: "V16"
    value {
      float_list {
        value: -2.8900833129882812
      }
    }

In [11]:
#STATISTICSGEN
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to C:\Users\w10\AppData\Local\Temp\tfx-interactive-2022-06-09T23_39_36.874953-zuupnqn2\StatisticsGen\statistics\2\Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to C:\Users\w10\AppData\Local\Temp\tfx-interactive-2022-06-09T23_39_36.874953-zuupnqn2\StatisticsGen\statistics\2\Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [12]:
context.show(statistics_gen.outputs['statistics'])

In [14]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to C:\Users\w10\AppData\Local\Temp\tfx-interactive-2022-06-09T23_39_36.874953-zuupnqn2\SchemaGen\schema\3\schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [15]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Amount',FLOAT,required,single,-
'V1',FLOAT,required,single,-
'V10',FLOAT,required,single,-
'V11',FLOAT,required,single,-
'V12',FLOAT,required,single,-
'V13',FLOAT,required,single,-
'V14',FLOAT,required,single,-
'V15',FLOAT,required,single,-
'V16',FLOAT,required,single,-


In [16]:
#EXAMPLE VALIDATOR
# El componente de canalización ExampleValidator identifica anomalías en el entrenamiento y la entrega de datos. Puede detectar diferentes clases de anomalías en los datos. Por ejemplo, puede:

# realizar comprobaciones de validez comparando estadísticas de datos con un esquema que codifica las expectativas del usuario
# Detecte el sesgo de servicio-entrenamiento comparando datos de entrenamiento y servicio.
# detectar la deriva de datos mirando una serie de datos.
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to C:\Users\w10\AppData\Local\Temp\tfx-interactive-2022-06-09T23_39_36.874953-zuupnqn2\ExampleValidator\anomalies\4\Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to C:\Users\w10\AppData\Local\Temp\tfx-interactive-2022-06-09T23_39_36.874953-zuupnqn2\ExampleValidator\anomalies\4\Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [17]:
#After ExampleValidator finishes running, we can visualize the anomalies as a table.
context.show(example_validator.outputs['anomalies'])

In [18]:
_autoencoder_constant_file = 'autoencoder_constant.py'


In [19]:
%%writefile {_autoencoder_constant_file}



NUMERICAL_FEATURES = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']



Overwriting autoencoder_constant.py


In [20]:
_autoencoder_transform_module_file = 'autoencoder_transform.py'

In [21]:
#TRANSFORM 
# El componente de canalización Transform TFX realiza la ingeniería de características en tf.Examples emitidos desde un componente ExampleGen , utilizando un esquema de datos creado por un componente SchemaGen , y emite tanto un modelo guardado como estadísticas sobre los datos previos y posteriores a la transformación. Cuando se ejecuta, el modelo guardado aceptará tf.Examples emitidos desde un componente ExampleGen y emitirá los datos de características transformados.

# Consume: tf.Examples de un componente ExampleGen y un esquema de datos de un componente SchemaGen.
# Emite: un componente de modelo guardado en un entrenador, estadísticas previas y posteriores a la transformación.

In [22]:
%%writefile {_autoencoder_transform_module_file }

import tensorflow as tf
import tensorflow_transform as tft
import pandas as pd 


import autoencoder_constant

NUMERICAL_FEATURES = autoencoder_constant.NUMERICAL_FEATURES
def t_name(key):
    """
    Rename the feature keys so that they don't clash with the raw keys when
    running the Evaluator component.
    Args:
    key: The original feature key
    Returns:
    key with '_xf' appended
    """
    return key + '_xf'

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
    inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
    Map from string feature key to transformed feature operations.
    """
    outputs = {}

    for key in NUMERICAL_FEATURES:
        # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
        outputs[key] = tft.scale_to_0_1(inputs[key])

    return outputs



Overwriting autoencoder_transform.py


In [23]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_autoencoder_transform_module_file))
#context.run(transform, enable_cache=True)

In [24]:
context.run(transform, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for 'c:\\Users\\w10\\Desktop\\ML_Pipelines\\TFX\\autoencoder_transform.py' (including modules: ['autoencoder_constant', 'autoencoder_trainer', 'autoencoder_transform']).
INFO:absl:User module package has hash fingerprint version f7206835b541e3011e6b35521ffbf6ab7c0e0be839e975a0ba8cfb0e3a6c34fa.
INFO:absl:Executing: ['c:\\Users\\w10\\Desktop\\ML_Pipelines\\env\\Scripts\\python.exe', 'C:\\Users\\w10\\AppData\\Local\\Temp\\tmptbqt30v5\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\w10\\AppData\\Local\\Temp\\tmpi66_ypkf', '--dist-dir', 'C:\\Users\\w10\\AppData\\Local\\Temp\\tmpbqxp_zd9']
INFO:absl:Successfully built user code wheel distribution at 'C:\\Users\\w10\\AppData\\Local\\Temp\\tfx-interactive-2022-06-09T23_39_36.874953-zuupnqn2\\_wheels\\tfx_user_code_Transform-0.0+f7206835b541e3011e6b35521ffbf6ab7c0e0be839e975a0ba8cfb0e3a6c34fa-py3-none-any.whl'; target user module is 'autoencoder_transform'.
INFO:absl:Full user mo

Instructions for updating:
Use ref() instead.


INFO:absl:Feature Amount has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V1 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V10 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V11 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V12 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V13 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V14 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V15 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V16 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V17 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V18 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V19 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V2 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V20 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature V21 has no shape. Setting to VarLenSparseTe

NotFoundError: Failed to create a directory: C:\Users\w10\AppData\Local\Temp\tfx-interactive-2022-06-09T23_39_36.874953-zuupnqn2\Transform\updated_analyzer_cache\5/tfx-interactive-2022-06-09T23_39_36.874953-zuupnqn2-CsvExampleGen-examples-1-Split-train-STAR-8ef41c8838b9c8c1b22ff6ebd874a555aa5b070bc645a7f8b73c2b03c40c3199; No such file or directory

In [23]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['.temp_path']

In [24]:
_autoencoder_trainer_module_file = 'autoencoder_trainer.py'

In [25]:
%%writefile {_autoencoder_trainer_module_file}
from typing import Dict, List, Text
import tensorflow as tf 
import tensorflow_transform as tft
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx 
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput
import autoencoder_constant


NUMERICAL_FEATURES = autoencoder_constant.NUMERICAL_FEATURES

_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in NUMERICAL_FEATURES
       },
**{feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for feature in NUMERICAL_FEATURES },
}

def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
    """Generates features and label for tuning/training.

    Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
        dataset to combine in a single batch

    Returns:
    A dataset that contains (features, indices) tuple where features is a
        dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    return data_accessor.tf_dataset_factory(
        file_pattern,
        tfxio.TensorFlowDatasetOptions(
            batch_size=batch_size, label_key=NUMERICAL_FEATURES),
        tf_transform_output.transformed_metadata.schema)



class Autoencoder(keras.Model):
  """ Deep learning model. This object let you create an autoencoder, you specify the number of units neurons and the number of hidden layers, for the decoder and the encoder, 
  the same number of hidden layers and units neurons is taken to create both. also you specify the mid layer and the input shape  

  Args:
      keras (object): Model inheritance
  """

  def __init__(self,input_shape,intermediary_layers,z_layer,name='Autoencoder'):
    """function wich initialize the object

    Args:
        input_shape (int): number of the input shape of the model
        intermediary_layers (List): number of units nuerons and hidden layers 
        z_layer (int): mid layer
        name (str, optional): description of the model. Defaults to 'Autoencoder'.
    """
    tf.random.set_seed(3)
    super(Autoencoder,self).__init__(name=name,)
    self.z_layer = z_layer
    self.intermediary_layers = intermediary_layers
    self.encoder = keras.models.Sequential([
                                            keras.layers.InputLayer(input_shape = input_shape),
                                            self.dense_layers(self.intermediary_layers),
                                            keras.layers.Dense(units = self.z_layer, activation = 'relu')

    ])
    self.decoder = keras.models.Sequential([
                                            self.dense_layers(reversed(self.intermediary_layers)),
                                            keras.layers.Dropout(0.2),
                                            keras.layers.Dense(units = input_shape,activation='sigmoid')
    ])

  def dense_layers(self,sizes):
    """ create the hidden layers

    Args:
        sizes (List): number of units nuerons and hidden layers 

    Returns:
        object: keras model wich contain the hidden layers
    """
    model = keras.Sequential()
    
    for size in sizes: 
      model.add( tf.keras.layers.Dense(size, activation='relu'))
      model.add(keras.layers.Dropout(0.1))
    
    return model                   

  def call(self,x):
    """ method wich let the call of the model

    Args:
        x (DataFrame, Array): input

    Returns:
        object: Model 
    """
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    
    return decoded

def build_keras_model():
    model = Autoencoder(len(NUMERICAL_FEATURES),intermediary_layers=[300,200,100,50,25],z_layer=30)

    model.compile(optimizer='adam',metrics=[keras.metrics.mean_squared_error], loss=keras.losses.MeanSquaredError())

    return model


def keras_train_and_evaluate(model, train_dataset, validation_dataset, epochs=10):

    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=5,restore_best_weights=True)
    history = model.fit(x=train_dataset, y=train_dataset,
                    epochs=100,
                    shuffle=True,
                    validation_data=(validation_dataset, validation_dataset),callbacks = [early_stopping] )
    #Add callbacks

    return model

# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=20)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=10)

  model = build_keras_model()
  model = keras_train_and_evaluate(model, train_dataset, eval_dataset, fn_args.custom_config['epochs'])

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')


Writing autoencoder_trainer.py


In [26]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_autoencoder_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for 'c:\\Users\\w10\\Desktop\\ML_Pipelines\\TFX\\autoencoder_trainer.py' (including modules: ['autoencoder_constant', 'autoencoder_trainer', 'autoencoder_transform']).
INFO:absl:User module package has hash fingerprint version f7206835b541e3011e6b35521ffbf6ab7c0e0be839e975a0ba8cfb0e3a6c34fa.
INFO:absl:Executing: ['c:\\Users\\w10\\Desktop\\ML_Pipelines\\env\\Scripts\\python.exe', 'C:\\Users\\w10\\AppData\\Local\\Temp\\tmppfq7025w\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\w10\\AppData\\Local\\Temp\\tmp8d1bdq3e', '--dist-dir', 'C:\\Users\\w10\\AppData\\Local\\Temp\\tmp9rs8_ic_']
INFO:absl:Successfully built user code wheel distribution at 'C:\\Users\\w10\\AppData\\Local\\Temp\\tfx-interactive-2022-06-09T16_03_54.503872-vrfk689q\\_wheels\\tfx_user_code_Trainer-0.0+f7206835b541e3011e6b35521ffbf6ab7c0e0be839e975a0ba8cfb0e3a6c34fa-py3-none-any.whl'; target user module is 'autoencoder_trainer'.
INFO:absl:Full user module p

AttributeError: transformed_metadata

In [33]:
def _create_pipeline(pipeline_name: str, root_path: str, data_root: str,
                     module_train_file: str,module_transform_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  
    _pipeline_root = os.path.join(root_path, 'pipelines');      # Join ~/tfx/pipelines/
    _metadata_db_root = os.path.join(root_path, 'metadata.db');    # Join ~/tfx/metadata.db
    _log_root = os.path.join(root_path, 'logs');
    _model_root = os.path.join(root_path, 'model');
    _serving_model_dir = os.path.join(root_path, 'serving_model')
    # Brings data into the pipeline.
    example_gen = tfx.components.CsvExampleGen(input_base=data_root)
    statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'])
    schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs['statistics'],infer_feature_shape=False)
    example_validator = tfx.components.ExampleValidator(statistics=statistics_gen.outputs['statistics'],schema=schema_gen.outputs['schema'])
    transform = tfx.components.Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        module_file=os.path.abspath(module_transform_file))
    trainer = tfx.components.Trainer(
        module_file=os.path.abspath(module_train_file),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=tfx.proto.TrainArgs(num_steps=10000),
        eval_args=tfx.proto.EvalArgs(num_steps=5000))

    # Pushes the model to a filesystem destination.
    pusher = tfx.components.Pusher(
        model=trainer.outputs['model'],
        push_destination=tfx.proto.PushDestination(
            filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
    components = [
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        trainer,
        pusher,
    ]

    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=_pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata
        .sqlite_metadata_connection_config(metadata_path),
        components=components)

In [37]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name='autoencoder_local_pipeline',
      root_path=_autoencoder_root,
      data_root=_data_root,
      module_transform_file=_autoencoder_transform_module_file,
      module_train_file=_autoencoder_trainer_module_file,
      serving_model_dir=_serving_model_dir,
      metadata_path=_metadata))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for 'c:\\Users\\w10\\Desktop\\ML_Pipelines\\TFX\\autoencoder_transform.py' (including modules: ['autoencoder_constant', 'autoencoder_trainer', 'autoencoder_transform']).
INFO:absl:User module package has hash fingerprint version f7206835b541e3011e6b35521ffbf6ab7c0e0be839e975a0ba8cfb0e3a6c34fa.
INFO:absl:Executing: ['c:\\Users\\w10\\Desktop\\ML_Pipelines\\env\\Scripts\\python.exe', 'C:\\Users\\w10\\AppData\\Local\\Temp\\tmp9h2m_enu\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\w10\\AppData\\Local\\Temp\\tmpgop_yw_e', '--dist-dir', 'C:\\Users\\w10\\AppData\\Local\\Temp\\tmpinc5eb45']
INFO:absl:Successfully built user code wheel distribution at 'c:\\Users\\w10\\Desktop\\ML_Pipelines\\env\\lib\\site-packages\\tfx\\v1\\examples/auto

InvalidArgumentError: Failed to create a directory: c:\Users\w10\Desktop\ML_Pipelines\env\lib\site-packages\tfx\v1\examples/autoencoder_pipeline\pipelines\CsvExampleGen\.system/stateful_working_dir/2022-06-09T17:43:50.990248; Invalid argument